In [2]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

from matplotlib import rcParams
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['font.family'] = 'serif'
rcParams['mathtext.fontset'] = 'cm'
rcParams['figure.dpi'] = 100
rcParams['legend.fontsize'] = 10
rcParams['axes.labelsize'] = 'large'

In [120]:
g = nx.DiGraph()
g.add_nodes_from([1, 2, 3, 4, 5, 6])
g.add_edges_from([(1, 2), (1, 3), (1, 6),
                  (2, 3), (2, 4), (2, 5),
                  (3, 6)])
A = nx.to_scipy_sparse_matrix(g).T  # Matriz de Adyacencia
One = np.ones(6)
k_out = One.T@A  # Vector Grado Out
k_in = A@One  # Vector Grado In
D = np.diag(k_in)
L = D - A  # Laplaciano

/tmp/ipykernel_7112/476097011.py:6: DeprecationWarning: 

The scipy.sparse array containers will be used instead of matrices
in Networkx 3.0. Use `to_scipy_sparse_array` instead.
  A = nx.to_scipy_sparse_matrix(g).T  # Matriz de Adyacencia


In [127]:
Ac = A.copy()
for j in range(2, 4):
    Ac += A**j
    print(np.sum(Ac.astype(bool), axis=0))


[[5 4 1 0 0 0]]
[[5 4 1 0 0 0]]


In [130]:
(A@A == A**2)

/home/agus/mambaforge/envs/facu39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


<6x6 sparse matrix of type '<class 'numpy.bool_'>'
	with 36 stored elements in Compressed Sparse Column format>